In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

import datetime
import plotly.graph_objects as go
from ipywidgets import widgets



In [2]:
pd.read_csv('../raw_data/LIGA/season-1415_csv.csv').loc[:, 'Div':'BbAH']

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH
0,SP1,23/08/14,Almeria,Espanol,1,1,D,0,0,D,...,3.35,3.18,2.90,2.75,44,2.34,2.23,1.70,1.64,26
1,SP1,23/08/14,Granada,La Coruna,2,1,H,0,1,A,...,3.45,3.23,4.40,3.99,36,2.61,2.43,1.61,1.54,24
2,SP1,23/08/14,Malaga,Ath Bilbao,1,0,H,1,0,H,...,3.40,3.23,2.80,2.55,45,2.14,2.02,1.85,1.77,26
3,SP1,23/08/14,Sevilla,Valencia,1,1,D,1,0,H,...,3.65,3.41,4.00,3.66,44,1.97,1.88,2.00,1.92,23
4,SP1,24/08/14,Barcelona,Elche,3,0,H,1,0,H,...,14.01,10.13,36.00,26.00,35,1.31,1.29,3.85,3.48,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,SP1,23/05/15,Granada,Ath Madrid,0,0,D,0,0,D,...,2.50,2.37,2.60,2.39,36,2.50,2.33,1.67,1.59,25
376,SP1,23/05/15,Levante,Elche,0,0,D,0,0,D,...,3.62,3.42,4.10,3.83,40,2.11,1.99,1.87,1.81,26
377,SP1,23/05/15,Malaga,Sevilla,2,3,A,0,0,D,...,4.00,3.73,2.21,2.13,37,1.60,1.48,2.70,2.60,25
378,SP1,23/05/15,Real Madrid,Getafe,7,3,H,3,3,D,...,12.00,9.49,29.00,20.36,29,1.22,1.18,5.40,4.51,22


In [73]:
league = ['BundesLiga','LIGA','Ligue1','PL','SerieA']

In [3]:
#directory_in_str = '/Users/antoinelemaire/code/Zorix9/beat_the_bookies/raw_data/LIGA'
directory_in_str = '../raw_data/LIGA'
directory = os.fsencode(directory_in_str)

list_df = [ pd.read_csv(directory_in_str+'/'+os.fsdecode(file)) for file in os.listdir(directory) if os.fsdecode(file).endswith('.csv')]
#print(f'{count} seasons in the dataframe')

In [109]:
list_df[0][('HomeTeam')].unique()

array(['Betis', 'Girona', 'Barcelona', 'Celta', 'Villarreal', 'Eibar',
       'Real Madrid', 'Vallecano', 'Ath Bilbao', 'Valencia', 'Getafe',
       'Leganes', 'Alaves', 'Ath Madrid', 'Valladolid', 'Espanol',
       'Sevilla', 'Levante', 'Huesca', 'Sociedad'], dtype=object)

In [89]:
for df in list_df:
    df['FTR_home'] = df['FTR'].apply(lambda x: 'Win' if str(x) == 'H' else ('Lose' if str(x) =='A' else 'Draw'))
    df['FTR_away'] = df['FTR'].apply(lambda x: 'Lose' if str(x) == 'H' else ('Win' if str(x)=='A' else 'Draw'))

In [111]:
list_df[0].head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,FTR_home,FTR_away
0,SP1,17/08/2018,Betis,Levante,0,3,A,0,1,A,...,-0.75,1.89,1.85,2.07,2.00,1.59,4.42,5.89,Lose,Win
1,SP1,17/08/2018,Girona,Valladolid,0,0,D,0,0,D,...,-0.75,2.06,2.01,1.90,1.85,1.76,3.57,5.62,Draw,Draw
2,SP1,18/08/2018,Barcelona,Alaves,3,0,H,0,0,D,...,-2.50,1.95,1.91,2.00,1.95,1.10,11.85,32.17,Win,Lose
3,SP1,18/08/2018,Celta,Espanol,1,1,D,0,1,A,...,-0.75,2.26,2.18,1.74,1.71,2.18,3.26,3.85,Draw,Draw
4,SP1,18/08/2018,Villarreal,Sociedad,1,2,A,1,1,D,...,-0.25,1.76,1.74,2.23,2.14,2.32,3.21,3.53,Lose,Win


# 1. Visualisation

In [7]:
team_stats = widgets.Dropdown(
    description='Teams:   ',
    value='Ath Madrid',
    options=list_df[4]['HomeTeam'].unique().tolist()
)


# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['FTR_home'], opacity=0.75, name='Result at home over the year')
trace2 = go.Histogram(x=df['FTR_away'], opacity=0.75, name='Result away over the year')
g = go.FigureWidget(data=[trace1,trace2],
                    layout=go.Layout(
                        title=dict(
                            text='Team result'
                        ),
                        barmode='overlay'
                    ))

In [8]:
def validate():
    if team_stats.value in list_df[4]['HomeTeam'].unique():
        return True
    else:
        return False

def response(change):
    if validate():
        filter_list = [i for i in
                        df['HomeTeam'] == team_stats.value]
        temp_df = df[filter_list]
        x1 = temp_df['FTR_home']
        x2 = temp_df['FTR_away']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Result'
            g.layout.yaxis.title = 'Number of matchs'    

In [9]:
team_stats.observe(response, names='value')

In [10]:
container2 = widgets.HBox([team_stats])
widgets.VBox([container2,
              g])

# 1.1 Visualisation that works better


In [11]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,FTR_home,FTR_away
0,SP1,21/08/15,Malaga,Sevilla,0,0,D,0,0,D,...,0.25,2.02,1.95,1.94,1.90,3.22,3.50,2.36,Draw,Draw
1,SP1,22/08/15,Ath Madrid,Las Palmas,1,0,H,1,0,H,...,-1.75,2.02,1.95,1.95,1.89,1.25,6.24,16.90,Win,Lose
2,SP1,22/08/15,Espanol,Getafe,1,0,H,1,0,H,...,-0.50,1.95,1.90,2.01,1.94,2.10,3.28,4.19,Win,Lose
3,SP1,22/08/15,La Coruna,Sociedad,0,0,D,0,0,D,...,0.00,1.82,1.78,2.15,2.07,2.44,3.41,3.15,Draw,Draw
4,SP1,22/08/15,Vallecano,Valencia,0,0,D,0,0,D,...,0.50,1.90,1.83,2.08,2.01,2.93,3.68,2.46,Draw,Draw


In [116]:
list_season = ['09-10','10-11','11-12','12-13','13-14','14-15','15-16','16-17','17-18','18-19']

In [132]:
def bar_plt(season, team):
    ## season choice 
    dico = dict(zip(list_season[::-1],range(0,10)))
    df = list_df[dico[season]]
    ## for the home games
    try:
        filter_list = [i for i in
                        df['HomeTeam'] == team]
        temp_df = df[filter_list]
        x1 = temp_df['FTR_home']

        ## for the away games
        filter_list_away = [i for i in
                        df['AwayTeam'] == team]
        temp_df_away = df[filter_list_away]
        x2 = temp_df_away['FTR_away']

        x = ['Win','Draw','Lose']
        x_axis = np.arange(3)

        plt.bar(x_axis + 0.21,x1.value_counts()[['Win','Draw','Lose']],0.4,label='Home Games')
        plt.bar(x_axis - 0.21,x2.value_counts()[['Win','Draw','Lose']],0.4,label='Away Games')


        plt.xticks(x_axis, x)
        plt.xlabel("Result")
        plt.ylabel("Number of matches")
        plt.title(f"Result of {team} over the season {season}")
        plt.legend()
        plt.ylim([0,17])
    except:
        print(f'{team} wasnt playing during the year {season}')
    #plt.show()
    
    

In [156]:
widgets.interact(scatter_plt,season=list_season,team=list(df['HomeTeam'].unique()));
#widgets.interact(bar_plt,season=list_season,team=['Ath Madrid','Real Madrid']);

interactive(children=(Dropdown(description='season', options=('09-10', '10-11', '11-12', '12-13', '13-14', '14…

# 1.2 Statistics of the teams

In [134]:
for df in list_df:
    df['Home_point'] = df['FTR'].apply(lambda x: 3 if str(x) == 'H' else (0 if str(x) =='A' else 1))
    df['Away_point'] = df['FTR'].apply(lambda x: 0 if str(x) == 'H' else (3 if str(x)=='A' else 1))

## 1.2.1 test classement

In [135]:
df1 = list_df[1]
df1.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,FTR_home,FTR_away,Home_point,Away_point
0,SP1,18/08/12,Celta,Malaga,0,1,A,0,0,D,...,1.96,1.98,1.91,2.35,3.45,3.29,Lose,Win,0,3
1,SP1,18/08/12,Mallorca,Espanol,2,1,H,1,1,D,...,1.98,1.95,1.90,2.25,3.33,3.63,Win,Lose,3,0
2,SP1,18/08/12,Sevilla,Getafe,2,1,H,2,0,H,...,1.82,2.12,2.05,1.68,3.96,5.73,Win,Lose,3,0
3,SP1,19/08/12,Ath Bilbao,Betis,3,5,A,0,3,A,...,2.12,1.81,1.76,2.17,3.49,3.65,Lose,Win,0,3
4,SP1,19/08/12,Barcelona,Sociedad,5,1,H,4,1,H,...,1.93,1.99,1.93,1.09,13.96,35.36,Win,Lose,3,0


In [141]:
lst_points = []
for item in df1['HomeTeam'].unique():
    lst_points.append(df1[df1['HomeTeam']==item]['Home_point'].sum() + df1[df1['AwayTeam']==item]['Away_point'].sum())
lst_points

[37,
 36,
 50,
 45,
 100,
 46,
 85,
 35,
 53,
 34,
 56,
 44,
 57,
 66,
 47,
 42,
 39,
 65,
 76,
 43]

In [143]:
classement = pd.DataFrame(list(zip(df1['HomeTeam'].unique(),lst_points)),columns=['team','points'])

In [152]:
classement = classement.sort_values(by='points',ascending=False).reset_index(drop=True)

## 1.2.2 Fonction classement 

In [166]:
def classment_per_year():
    '''
    Compute the classement of every season 
    ATTENTION : fct prend rien en paarmètre mais utilise la variable list_df
    '''
    list_df_classement = []
    for df in list_df:
        lst_points = []
        
        for item in df['HomeTeam'].unique():
           lst_points.append(df[df['HomeTeam']==item]['Home_point'].sum() + df[df['AwayTeam']==item]['Away_point'].sum())
        
        classement = pd.DataFrame(list(zip(df1['HomeTeam'].unique(),lst_points)),columns=['team','points'])
        classement = classement.sort_values(by='points',ascending=False).reset_index(drop=True)
        list_df_classement.append(classement)
    return list_df_classement

In [168]:
test_classement = classment_per_year()

In [171]:
test_classement[2]

,team,points
0,Granada,93
1,Sevilla,90
2,Levante,78
3,Barcelona,72
4,Valladolid,67
5,Real Madrid,64
6,Valencia,63
7,Espanol,56
8,Osasuna,55
9,Malaga,54


In [172]:
dic_season ={'09-10':9,'10-11':8,'11-12':7,'12-13':6,'13-14':5,'14-15':4,'15-16':3,'16-17':2,'17-18':1,'18-19':0}

In [173]:
list(dic_season.keys())

['09-10',
 '10-11',
 '11-12',
 '12-13',
 '13-14',
 '14-15',
 '15-16',
 '16-17',
 '17-18',
 '18-19']